In [1]:
%tensorflow_version 2.x

In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-11-23 13:33:51--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-11-23 13:33:52--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc954aad8736914609f04e213150.dl.dropboxusercontent.com/zip_download_get/AnLHXYw5Olqrkies0Udxoc3DvChoY1QmaG8tEoN81szsYfILMh7c6Wu5LU4M_aWuPBr3etP7yj06bODVXkmVyOSe8TbPj9Qe_aN9a5ybhVx7-w [following]
--2020-11-23 13:33:52--  https://uc954aad8736914609f04e213150.dl.dropboxusercontent.com/zip_download_get/AnLHXYw5Olqrkies0Udxoc3DvChoY1QmaG8tEoN81szsYfILMh7c6Wu5LU4M_aWuPBr3etP7yj06bODVXkmV

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [4]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')

In [ ]:
filter

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [5]:
filter.shape

(1306124, 3)

In [6]:
filter.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [7]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , What are kinds of Swedish are spoken today except standard Swedish?                                                       1
If a war broke out between either sides of prime meridian (Greenwich) , who would win?                                    1
Why is it that sometimes people don't know when to shut up?                                                               1
What is the main difference between Swift and wires? Does Swift make actual fund transfer or just a messaging service?    1
What would a force field look like?                                                                                       1
                                                                                                                         ..
What is Gertrude Caton Thompson known for?                                                                                1
What do I do I have a girlfriend but I like touching another girl?                                                  

In [8]:
filter.drop([1306122,1306123],0,inplace=True)

In [9]:
filter.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

In [10]:
filter.shape

(1306122, 3)

In [11]:
filter.drop(['PK'],1,inplace=True)

In [12]:
filter.dtypes

question_text     object
target           float64
dtype: object

In [13]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [14]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-11-23 13:34:57--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-11-23 13:34:57--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-11-23 13:34:57--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [15]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [16]:
!rm /content/glove.42B.300d.zip

In [17]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [18]:
len(embeding_index['provide'])

300

In [19]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [20]:
len(embeding_index)

1917494

In [21]:
X=filter['question_text']
Y=filter['target']

In [22]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [23]:
class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(class_weight)

[0.53303076 8.0687027 ]


In [24]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5330307596717635, 1: 8.068702702702703}

In [25]:
x_train

727408     Are top nlus suitable to study if I want to ta...
848250     What is a question that you hope your spouse/s...
1071911    Are there people who still wear braces in coll...
29391      What month do accounting years typically start...
555921            How people get attracted towards same sex?
                                 ...                        
1184280    I am trying to make a program in Python that w...
561637     Do the Scottish people think the 307 years of ...
1043905                     Should I give UPSC 2017 attempt?
1275007                                  Who is Nills Bildt?
97905      Are there any Robotics/mechatronics engineers ...
Name: question_text, Length: 1044897, dtype: object

In [26]:
from nltk import word_tokenize

In [27]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [29]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [30]:
np.percentile(sent_lens,99)

44.0

In [31]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [32]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'get': 32,
 'when': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'they': 46,
 'by': 47,
 'was': 48,
 'did': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'between': 68,
 'many': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [33]:
vocab_size

195854

In [34]:
seq_train_matrix.shape

(1044897, 44)

In [35]:
seq_train

[[12,
  270,
  21236,
  1901,
  5,
  187,
  20,
  8,
  85,
  5,
  86,
  82,
  13973,
  36,
  8,
  24,
  1049,
  19,
  39,
  12,
  69,
  590,
  84,
  196,
  78,
  1,
  186,
  114,
  10,
  216,
  6244,
  32,
  70,
  5,
  276,
  340],
 [2, 3, 4, 304, 19, 14, 2202, 28, 1909, 1541, 73, 233, 2535, 14],
 [12, 39, 37, 40, 129, 508, 5230, 6, 145],
 [2, 467, 11, 2146, 114, 2979, 120, 10, 323, 13, 1937],
 [9, 37, 32, 1575, 620, 141, 169],
 [20,
  14,
  97,
  14368,
  1,
  2371,
  341,
  7,
  2075,
  102,
  1043,
  27,
  2379,
  5,
  323,
  136,
  1,
  280,
  7,
  411,
  23,
  114,
  44,
  94,
  56,
  3459,
  9,
  35,
  14,
  11,
  17],
 [2, 12, 1, 383, 7, 14566, 22, 354],
 [41, 8, 105, 5, 1859, 20, 119, 1351, 5, 5742],
 [9, 31, 8, 880, 1, 46628, 17969],
 [2, 12, 1, 2135, 7, 234, 53390],
 [15, 8, 62, 4, 262, 13, 4, 167, 43, 437, 114, 135],
 [49, 50, 617, 1710, 6776, 577, 6, 1, 1179, 229],
 [16,
  26,
  2544,
  129,
  24,
  72,
  7,
  154,
  937,
  111,
  22,
  6065,
  589,
  71,
  4686,
  150,
  1

In [36]:
x_train.shape

(1044897,)

In [37]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [38]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [39]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [40]:
 len(embedding_matrix)

195855

In [41]:
embedding_matrix.shape

(195855, 300)

In [42]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.33566999, -0.40494999, -0.091062  , ..., -0.27849999,
        -0.58424002, -0.1167    ],
       [ 0.059442  ,  0.13139001,  0.19475   , ..., -0.58645999,
        -0.22643   ,  0.57139999]])

In [43]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [44]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58756500  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [45]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [46]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [47]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [48]:
model.fit(seq_train_matrix,y_train,
              epochs=8,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=10000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/8
105/105 [==============================] - ETA: 0s - loss: 0.3426 - accuracy: 0.8453
Epoch 00001: val_loss improved from inf to 0.25691, saving model to /content/weights-01-0.2569.h5
105/105 [==============================] - 173s 2s/step - loss: 0.3426 - accuracy: 0.8453 - val_loss: 0.2569 - val_accuracy: 0.8937
Epoch 2/8
105/105 [==============================] - ETA: 0s - loss: 0.2842 - accuracy: 0.8790
Epoch 00002: val_loss did not improve from 0.25691
105/105 [==============================] - 177s 2s/step - loss: 0.2842 - accuracy: 0.8790 - val_loss: 0.3015 - val_accuracy: 0.8680
Epoch 3/8
105/105 [==============================] - ETA: 0s - loss: 0.2623 - accuracy: 0.8878
Epoch 00003: val_loss did not improve from 0.25691
105/105 [==============================] - 177s 2s/step - loss: 0.2623 - accuracy: 0.8878 - val_loss: 0.2776 - val_accuracy: 0.8886
Epoch 4/8
105/105 [==============================] - ETA: 0s - loss: 0.2473 - accuracy: 0.8925
Epoch 00004: val_loss im

In [51]:
best_model = load_model('/content/weights-04-0.2386.h5')

In [ ]:
best_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 44)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 44, 300)           58758600  
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               3289

In [52]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9609791966926791